# Step01: Data Cleaning
In this notebook, the telecomunications churn data is imported and cleaned.  This data originates from https://www.kaggle.com/becksddf/churn-in-telecoms-dataset

## Import required packages

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

## Import csv file and create a copy for transformations

In [2]:
churn_og = pd.read_csv('../data/telecoms_churn_raw.csv')
churn = churn_og.copy()

## Replace ' ' with '_' in the column names

In [3]:
churn.columns = churn.columns.str.replace(' ', '_')

In [4]:
churn.head()

,state,account_length,area_code,phone_number,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,...,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,customer_service_calls,churn
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


## Replace values in the churn column (target variable)
- Repace True with 1
- Replace False with 0

In [5]:
churn.churn.value_counts()

False    2850
True      483
Name: churn, dtype: int64

In [6]:
churn.churn = churn_og.churn.apply(lambda c: 1 if c == True else 0)

In [7]:
churn.churn.value_counts()

0    2850
1     483
Name: churn, dtype: int64

### Key Takeaway
The True or 1 class, is smaller than the False or 0 class.  Should consider __SMOTE__ function prior to train, test, split


## Review dtypes

In [8]:
churn.dtypes

state                      object
account_length              int64
area_code                   int64
phone_number               object
international_plan         object
voice_mail_plan            object
number_vmail_messages       int64
total_day_minutes         float64
total_day_calls             int64
total_day_charge          float64
total_eve_minutes         float64
total_eve_calls             int64
total_eve_charge          float64
total_night_minutes       float64
total_night_calls           int64
total_night_charge        float64
total_intl_minutes        float64
total_intl_calls            int64
total_intl_charge         float64
customer_service_calls      int64
churn                       int64
dtype: object

## Review each column in turn

### state
- Candidate for categorical variable later on, will create lots of additional columns
- Not all states are represented equally in the data set
    - Max is WV with 106
    - Min is CA with 34
- Looking ahead, the area codes included in this data set include San Francisco, San Jose, and Oakland.  These are all located in CA.  When applying states numberical identifiers, CA will equal 0 and remaining states are assigned 1-51 based on alphabetical order.

In [9]:
churn.state.describe()

count     3333
unique      51
top         WV
freq       106
Name: state, dtype: object

In [10]:
churn.state.value_counts();

In [11]:
###  In order to ID each state with a numeric value, the state column is captured in aphabetical order.
###  Then CA is inserted as item 0 and popped from the list.  Finally this list is enumerated.
# sorted_states = churn.state.sort_values().unique().tolist()
# sorted_states.insert(0, 'CA')
# sorted_states.pop(5)
# sorted_states = list(enumerate(sorted_states))

In [12]:
# state_id = {}

# for s in sorted_states:
#     state_id.update({s[1]: s[0]})

In [13]:
# churn.state = churn_og['state'].apply(lambda s: state_id[s])

In [14]:
churn.state.value_counts();

### account_length
- Assuming the value indiates the age of the account

In [15]:
churn.account_length.describe()

count    3333.000000
mean      101.064806
std        39.822106
min         1.000000
25%        74.000000
50%       101.000000
75%       127.000000
max       243.000000
Name: account_length, dtype: float64

### area_code
- Three total values
- 415 represents 1655 rows
- The other two area codes are roughly half
    - 510 represents 840 rows
    - 408 represents 838 rows

In [16]:
churn.area_code.describe()

count    3333.000000
mean      437.182418
std        42.371290
min       408.000000
25%       408.000000
50%       415.000000
75%       510.000000
max       510.000000
Name: area_code, dtype: float64

In [17]:
churn.area_code.value_counts()

415    1655
510     840
408     838
Name: area_code, dtype: int64

In [18]:
churn.area_code = churn_og['area code'].apply(lambda a: str(a))

### phone_number
- All unique values most likely since the phone number prefix (first three numbers) and line number (last four numbers) are inlcude together and interpereted as a string
- Separating prefix and line numbers to investigate any categorical relationships yields:
    - Phone prefix values total 96 discrete values
        - Will keep for the time being, may eventually drop this column
    - Phone line numbers are completly unique
        - These provide no value in that they are completely unique, dropping from the dataframe

In [19]:
churn.phone_number.describe()

count         3333
unique        3333
top       334-9806
freq             1
Name: phone_number, dtype: object

In [20]:
churn.phone_number

0       382-4657
1       371-7191
2       358-1921
3       375-9999
4       330-6626
          ...   
3328    414-4276
3329    370-3271
3330    328-8230
3331    364-6381
3332    400-4344
Name: phone_number, Length: 3333, dtype: object

In [21]:
churn_og['phone number'].apply(lambda p: p[0:3]).describe()

### Could be interesting to include, will add to the dataframe

count     3333
unique      96
top        405
freq        53
Name: phone number, dtype: object

In [22]:
### Adding phone_prefix to dataframe
churn['phone_prefix'] = churn_og['phone number'].apply(lambda p: str(p[0:3]))

In [23]:
churn_og['phone number'].apply(lambda p: p[4:]).describe()

### Adds no value and will not be added to the dataframe

count     3333
unique    3333
top       4968
freq         1
Name: phone number, dtype: object

In [24]:
churn = churn.drop(columns = ['phone_number'])

### international_plan
- Binary yes/no
- Candidate for categorical variable
- Far more no values (3010) then yes values (323)

In [25]:
churn.international_plan.describe()

count     3333
unique       2
top         no
freq      3010
Name: international_plan, dtype: object

In [26]:
churn.international_plan.value_counts()

no     3010
yes     323
Name: international_plan, dtype: int64

In [27]:
# churn['international_plan'] = churn_og['international plan'].apply(lambda i: 0 if i == 'no' else 1)
# churn.international_plan.value_counts()

### voice_mail_plan
- Binary yes/no
- More no values (2411) than yes values (922)

In [28]:
churn.voice_mail_plan.describe()

count     3333
unique       2
top         no
freq      2411
Name: voice_mail_plan, dtype: object

In [29]:
churn.voice_mail_plan.value_counts()

no     2411
yes     922
Name: voice_mail_plan, dtype: int64

In [30]:
# churn['voice_mail_plan'] = churn_og['voice mail plan'].apply(lambda i: 0 if i == 'no' else 1)
# churn.voice_mail_plan.value_counts()

### number_vmail_messages
- Integers ranging from zero to 51 voicemails
- Candidate for binning these values

In [31]:
churn.number_vmail_messages.describe()

count    3333.000000
mean        8.099010
std        13.688365
min         0.000000
25%         0.000000
50%         0.000000
75%        20.000000
max        51.000000
Name: number_vmail_messages, dtype: float64

In [32]:
churn.number_vmail_messages.min(), churn.number_vmail_messages.max()

(0, 51)

In [33]:
churn.number_vmail_messages.value_counts();

### day metrics (total_day_minutes, total_day_calls, total_day_charge)
- Candidates for binning

In [34]:
day = churn[['total_day_minutes', 'total_day_calls', 'total_day_charge']]

In [35]:
day.dtypes

total_day_minutes    float64
total_day_calls        int64
total_day_charge     float64
dtype: object

In [36]:
day.describe()

,total_day_minutes,total_day_calls,total_day_charge
count,3333.000000,3333.000000,3333.000000
mean,179.775098,100.435644,30.562307
std,54.467389,20.069084,9.259435
min,0.000000,0.000000,0.000000
25%,143.700000,87.000000,24.430000
50%,179.400000,101.000000,30.500000
75%,216.400000,114.000000,36.790000
max,350.800000,165.000000,59.640000


### eve metrics (total_eve_minutes, total_eve_calls, total_eve_charge)

In [37]:
eve = churn[['total_eve_minutes', 'total_eve_calls', 'total_eve_charge']]

In [38]:
eve.dtypes

total_eve_minutes    float64
total_eve_calls        int64
total_eve_charge     float64
dtype: object

In [39]:
eve.describe()

,total_eve_minutes,total_eve_calls,total_eve_charge
count,3333.000000,3333.000000,3333.000000
mean,200.980348,100.114311,17.083540
std,50.713844,19.922625,4.310668
min,0.000000,0.000000,0.000000
25%,166.600000,87.000000,14.160000
50%,201.400000,100.000000,17.120000
75%,235.300000,114.000000,20.000000
max,363.700000,170.000000,30.910000


### night metrics (total_night_minutes, total_night_calls, total_eve_charge)

In [40]:
night = churn[['total_night_minutes', 'total_night_calls', 'total_night_charge']]

In [41]:
night.dtypes

total_night_minutes    float64
total_night_calls        int64
total_night_charge     float64
dtype: object

In [42]:
night.describe()

,total_night_minutes,total_night_calls,total_night_charge
count,3333.000000,3333.000000,3333.000000
mean,200.872037,100.107711,9.039325
std,50.573847,19.568609,2.275873
min,23.200000,33.000000,1.040000
25%,167.000000,87.000000,7.520000
50%,201.200000,100.000000,9.050000
75%,235.300000,113.000000,10.590000
max,395.000000,175.000000,17.770000


### intl metrics (total_intl_minutes, total_intl_calls, total_intl_charge)

In [43]:
intl = churn[['total_intl_minutes', 'total_intl_calls', 'total_intl_charge']]

In [44]:
intl.dtypes

total_intl_minutes    float64
total_intl_calls        int64
total_intl_charge     float64
dtype: object

In [45]:
intl.describe()

,total_intl_minutes,total_intl_calls,total_intl_charge
count,3333.000000,3333.000000,3333.000000
mean,10.237294,4.479448,2.764581
std,2.791840,2.461214,0.753773
min,0.000000,0.000000,0.000000
25%,8.500000,3.000000,2.300000
50%,10.300000,4.000000,2.780000
75%,12.100000,6.000000,3.270000
max,20.000000,20.000000,5.400000


### Adding new columns that total day, eve, night, and intl metrics into totals.  Three new columns that total each metric as a total

In [46]:
churn['total_minutes'] = churn.total_day_minutes + \
                            churn.total_eve_minutes + \
                            churn.total_night_minutes + \
                            churn.total_intl_minutes

In [47]:
churn['total_calls'] = churn.total_day_calls + \
                            churn.total_eve_calls + \
                            churn.total_night_calls + \
                            churn.total_intl_calls

In [48]:
churn['total_charge'] = churn.total_day_charge + \
                            churn.total_eve_charge + \
                            churn.total_night_charge + \
                            churn.total_intl_charge

In [49]:
churn[['total_minutes', 'total_calls', 'total_charge']];

### customer_service_calls
- Most common vlaue is 1 with 1,181 instances
- Only 13 greater than or equal to 7

In [50]:
churn.customer_service_calls.describe()

count    3333.000000
mean        1.562856
std         1.315491
min         0.000000
25%         1.000000
50%         1.000000
75%         2.000000
max         9.000000
Name: customer_service_calls, dtype: float64

In [51]:
churn.customer_service_calls.value_counts()

1    1181
2     759
0     697
3     429
4     166
5      66
6      22
7       9
9       2
8       2
Name: customer_service_calls, dtype: int64

## Export data to csv

In [52]:
churn.dtypes

state                      object
account_length              int64
area_code                  object
international_plan         object
voice_mail_plan            object
number_vmail_messages       int64
total_day_minutes         float64
total_day_calls             int64
total_day_charge          float64
total_eve_minutes         float64
total_eve_calls             int64
total_eve_charge          float64
total_night_minutes       float64
total_night_calls           int64
total_night_charge        float64
total_intl_minutes        float64
total_intl_calls            int64
total_intl_charge         float64
customer_service_calls      int64
churn                       int64
phone_prefix               object
total_minutes             float64
total_calls                 int64
total_charge              float64
dtype: object

In [53]:
churn.to_csv('../data/churn_cleaned.csv')